In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import zipfile

from sklearn import preprocessing, cross_validation
from sklearn.metrics import log_loss
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression

"""
Data fields

 Dates - timestamp of the crime incident
Category - category of the crime incident (only in train.csv). This is the target variable you are going to predict.
Descript - detailed description of the crime incident (only in train.csv)
 DayOfWeek - the day of the week
 PdDistrict - name of the Police Department District
Resolution - how the crime incident was resolved (only in train.csv)
Address - the approximate street address of the crime incident
X - Longitude
Y - Latitude
"""


#z1 = zipfile.ZipFile('train.zip')
#z2 = zipfile.ZipFile('test.zip')
train = pd.read_csv('train.csv', parse_dates = ['Dates'])
test = pd.read_csv('test.csv', parse_dates = ['Dates'])

d_District = dict()
d_day = dict()
d_time = dict()

for i in train.Dates.dt.hour:
    if i in d_time:
        d_time[i] += 1
    else:
        d_time[i] = 1

#plt.figure(1)
#plt.title("Crime time")
#plt.bar(d_time.keys(), d_time.values(), color='g')
#plt.show()

day_label = ['Monday', 'Tuesday', 'Friday', 'Wednesday', 'Thursday', 'Sunday', 'Saturday']
for day in train.DayOfWeek:
    if day in d_day:
        d_day[day] += 1
    else:
        d_day[day] = 1

#plt.figure(2)
#plt.bar(range(7), d_day.values(), align='center')
#plt.xticks(range(7), day_label)
#plt.show()

for location in train.PdDistrict:
    if location in d_District:
        d_District[location] += 1
    else:
        d_District[location] = 1


#Convert crime labels to numbers
le_crime = preprocessing.LabelEncoder()
crime = le_crime.fit_transform(train.Category)

days = pd.get_dummies(train.DayOfWeek)
district = pd.get_dummies(train.PdDistrict)
hour = train.Dates.dt.hour
hour = pd.get_dummies(hour) 

#Build new array and create train data and train label
train_data = pd.concat([hour, days, district], axis=1)
train_data['crime']=crime
crime_data = train_data.iloc[:,:-1]
crime_label = train_data['crime']

[crime_train_data, crime_test_data, crime_train_labels, crime_test_labels] = cross_validation.train_test_split(crime_data, crime_label, test_size=0.25)
#lgr = LogisticRegression(C=1e5)
#lgr.fit(crime_train_data, crime_train_labels.values.ravel())
#lgr.score(crime_test_data, crime_test_labels)


model = BernoulliNB()
model.fit(crime_train_data, crime_train_labels)
model.score(crime_test_data, crime_test_labels)


KeyError: 'crime'